In [13]:
import pandas as pd
csv_file1 = 'C:\\Users\\mthay00\\OneDrive - Blackhawk Network, Inc\\Documents\\project_folder\\int_2.csv'
df = pd.read_csv(csv_file1)
df.head()

Period   Channel  Number of Customers      TDV  Number of Orders  \
0  2023-01  Alchemer                    9  2280.80                26   
1  2023-02  Alchemer                    7  1276.22                14   
2  2023-03  Alchemer                    4   275.00                 5   
3  2023-04  Alchemer                    4  6712.04                 7   
4  2023-05  Alchemer                    5  7115.34                21   

   Number of Recipients  Number of Redemptions  Number of Expired Gifts  
0                  1496                    248                     1248  
1                   126                     83                       43  
2                   114                     52                       62  
3                   469                    464                        5  
4                   511                    501                       10

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
Cell In[11], line 53, in update_graph(
    selected_channels=['Alchemer', 'End of Survey', 'Gift Links', 'HubSpot', 'Marketo', 'Microsoft Teams', 'PointsJoy Auto Rewards', 'Q-One Tech', 'Qualtrics', 'Reward Gallery', 'Salesforce', 'SurveyMonkey', 'SurveySparrow', 'Talkable', 'Voxco'],
    selected_metric=None
)
     50     selected_channels = channels.tolist()
     52 # Pivot the data for the selected metric
---> 53 pivot_table = data.pivot_table(values=selected_metric, index=['Month'], columns=['Channel', 'Year'], aggfunc='sum').reset_index()
        pivot_table =     Month_  API_2023  API_2024  Embedded_2023  Embedded_2024  \
0        1    7289.0    4502.0           10.0           29.0   
1        2    4635.0   11113.0            6.0           14.0   
2        3   22335.0   11867.0           14.0           22.0   
3        4   10709.0

In [15]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from dash import Dash, dcc, html, Input, Output

# Assuming df1 is already defined and contains the data
data = df

# Process data to ensure month alignment and correct formatting
data['Period'] = pd.to_datetime(data['Period'], format='%Y-%m')
data['Month'] = data['Period'].dt.month
data['Year'] = data['Period'].dt.year

# Define the metrics to plot
metrics = ['Number of Customers', 'TDV', 'Number of Orders', 'Number of Recipients', 'Number of Redemptions', 'Number of Expired Gifts']

# Create the Dash application
app = Dash(__name__)

# Get unique channels from the data
channels = data['Channel'].unique()

# Define colors for channels to ensure consistency
channel_colors = px.colors.qualitative.Plotly[:len(channels)]

# Layout of the app
app.layout = html.Div([
    dcc.Dropdown(
        id='channel-dropdown',
        options=[{'label': channel, 'value': channel} for channel in channels],
        multi=True,
        value=channels.tolist()
    ),
    dcc.Dropdown(
        id='metric-dropdown',
        options=[{'label': metric, 'value': metric} for metric in metrics],
        value=metrics[0]
    ),
    dcc.Graph(id='metric-graph')
])

# Callback to update the graph based on selected channels and metric
@app.callback(
    Output('metric-graph', 'figure'),
    [Input('channel-dropdown', 'value'),
     Input('metric-dropdown', 'value')]
)
def update_graph(selected_channels, selected_metric):
    if not selected_channels:
        selected_channels = channels.tolist()
    
    # Pivot the data for the selected metric
    pivot_table = data.pivot_table(values=selected_metric, index=['Month'], columns=['Channel', 'Year'], aggfunc='sum').reset_index()
    pivot_table.columns = ['_'.join(map(str, col)).strip() if isinstance(col, tuple) else col for col in pivot_table.columns]
    
    # Initialize the figure
    fig = go.Figure()

    # Ensure 'Month' column is correctly named after pivot and flatten
    if 'Month' in pivot_table.columns:
        month_column_name = 'Month'
    elif 'Month_' in pivot_table.columns:
        month_column_name = 'Month_'
    else:
        raise KeyError(f"Could not find 'Month' column in pivot_table for metric: {selected_metric}")

    # Add traces for each selected channel
    for i, channel in enumerate(selected_channels):
        if f'{channel}_2023' in pivot_table.columns:
            fig.add_trace(go.Scatter(
                x=pivot_table[month_column_name],
                y=pivot_table[f'{channel}_2023'],
                mode='lines',
                name=f'{channel} 2023',
                line=dict(dash='dot', color=channel_colors[i % len(channel_colors)]),
                hovertemplate=f"Channel: {channel}<br>Year: 2023<br>Month: %{{x}}<br>{selected_metric}: %{{y}}"
            ))
        if f'{channel}_2024' in pivot_table.columns:
            fig.add_trace(go.Scatter(
                x=pivot_table[month_column_name],
                y=pivot_table[f'{channel}_2024'],
                mode='lines',
                name=f'{channel} 2024',
                line=dict(dash='solid', color=channel_colors[i % len(channel_colors)]),
                hovertemplate=f"Channel: {channel}<br>Year: 2024<br>Month: %{{x}}<br>{selected_metric}: %{{y}}"
            ))

    # Update the layout to set month labels and other configurations
    fig.update_layout(
        title=f'{selected_metric} by Integration (2023 vs 2024)',
        xaxis=dict(
            title='Month',
            tickmode='array',
            tickvals=list(range(1, 13)),
            ticktext=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
        ),
        yaxis=dict(title=selected_metric),
        legend_title='Channel',
        template='plotly'
    )

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
